# Optical Character Recognition

## An data overview (1ST step)

In [ ]:
import pandas as pd
all_data = pd.read_csv('mnist_avaliacao.csv', sep=',')
test_data = pd.read_csv('mnist_teste.csv', sep=',')

In [ ]:
# overview types
all_data.dtypes.value_counts()

In [ ]:
# take a look at the last columns 
all_data.dtypes.tail()

In [ ]:
# lets take a look at the 'class' balance. I had to rename to 'y_class', because class is already in use in python 
all_data.y_class

In [ ]:
# take a look at the minimum value avaible
all_data.iloc[:, :-1].min().value_counts()

In [ ]:
# take a look at the maximum value avaible
all_data.iloc[:, :-1].max().value_counts()

In [ ]:
# lets overview values
all_data.describe()

In [ ]:
# lets recover 1 line and try to draw
import seaborn as sns
import matplotlib.pyplot as plt
from work_methods import matrix_recover

# choose a number between 1 and 2050 to draw
to_recover = 5
square_reference = 28

# define a matrix and rules
matrix = matrix_recover(all_data, to_recover, square_reference)

# heatmap plot of matrix
plt.figure(figsize=(28, 28))
sns.heatmap(matrix, annot=True, fmt="d", cmap="YlGnBu")
plt.show()

## use the classifier methods without any input filter (2ND step)

In [ ]:
# global checkpoint
# python libraries
import optuna
import pandas as pd 

# python codes
from resultado import Fold
from avaliacao import Experimento
from metodo import ScikitLearnAprendizadoDeMaquina

file_name = 'mnist_avaliacao.csv'

# define classes
numbers_names = ['0','1','2','3','4','5','6','7','8','9']
numbers_values = [x for x in range(0,10,1)]
numbers = dict(zip(numbers_values,numbers_names))

# recover DataFrame
df_data = pd.DataFrame(pd.read_csv(file_name))

#gera o fold e p experimento
folds = Fold.gerar_k_folds(df_data,val_k=5,col_classe='y_class',
                            num_repeticoes=1,seed=1,
                            num_folds_validacao=3,num_repeticoes_validacao=2)

### Random Forrest Classifier 28x28

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from avaliacao import OtimizacaoObjetivoRandomForest

clf_rf = RandomForestClassifier(random_state=1)
ml_method = ScikitLearnAprendizadoDeMaquina(clf_rf)
exp = Experimento(folds,ml_method, OtimizacaoObjetivoRandomForest, num_trials=10,
                    sampler=optuna.samplers.TPESampler(seed=1, n_startup_trials=3))
exp.calcula_resultados()

In [ ]:
studdy_fold_0 = exp.studies_per_fold[0]
studdy_fold_0.trials_dataframe().sort_values("value",ascending=False)

In [ ]:
from work_methods import parameters_graph
parameters_graph(exp.studies_per_fold[0].trials)

In [ ]:
from work_methods import show_results
show_results("Random Forest 28x28", numbers, exp)

### Decision Tree Classifier 28x28

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from avaliacao import OtimizacaoObjetivoArvoreDecisao

clf_dtree = DecisionTreeClassifier(random_state=1)
ml_method = ScikitLearnAprendizadoDeMaquina(clf_dtree)
exp = Experimento(folds,ml_method, OtimizacaoObjetivoArvoreDecisao, num_trials=10,
                    sampler=optuna.samplers.TPESampler(seed=1, n_startup_trials=3))
exp.calcula_resultados()

In [ ]:
studdy_fold_0 = exp.studies_per_fold[0]
studdy_fold_0.trials_dataframe().sort_values("value",ascending=False)

In [ ]:
parameters_graph(exp.studies_per_fold[0].trials)

In [ ]:
show_results("Decision Tree 28x28", numbers, exp)